# Computate Smart Cloud Builder

## About the open source GPL3 license and copyright for this product

Copyright © 2024 Computate Limited Liability Company in Utah, USA

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.

ADDITIONAL TERMS

As stated in section 7. c) and e) of the GPL3 license, 
"you may supplement the terms of this License with terms," 
Computate has added the following additional terms to the license: 

  7 c) Prohibiting misrepresentation of the origin of that material, and
    requiring that modified versions of such material be marked in
    reasonable ways as different from the original version;

  7 e) Declining to grant rights under trademark law for use of some
    trade names, trademarks, or service marks;

Please do not redistribute this course until you have built your own platform with these tools, 
separate from the computate.org platform, and reconfigure your fork of this repo to deploy 
your own platform instead of the computate.org platform. 

QUESTIONS

For questions about this open source license, please contact our public mailing list at computate@group.computate.org


## Create a Keycloak auth scopes

Here is where we define the default Keycloak auth scopes for our application: 

- POST
- PATCH
- GET
- Admin
- SuperAdmin

In [ ]:
eval $(env SITE_NAMESPACE="$(oc project -q)" ./vars.py)

ACCESS_TOKEN=$(curl -k -s -X POST -d "username=admin" -d "grant_type=password" -d "client_id=admin-cli" \
  -d "password=$(oc -n keycloak get secret/keycloak-initial-admin -o jsonpath={.data.admin-password} | base64 -d)" \
  $AUTH_URL/realms/master/protocol/openid-connect/token | jq -r ".access_token")

declare -a arr=("POST" "PATCH" "GET" "DELETE" "$AUTH_SCOPE_ADMIN" "$AUTH_SCOPE_SUPER_ADMIN")
for AUTH_SCOPE in "${arr[@]}"
do
  curl -k -i -X POST -H "Content-Type: application/json" -H "Authorization: Bearer $ACCESS_TOKEN" \
    "$AUTH_URL/admin/realms/$AUTH_REALM/clients/$AUTH_CLIENT/authz/resource-server/scope" \
    --data '{"id":"'"$AUTH_REALM-$AUTH_SCOPE"'","name":"'"$AUTH_SCOPE"'","displayName":"'"$AUTH_SCOPE"'"}'
done

echo DONE

## Create a Keycloak auth resource and permission for SiteUsers


Create a Keycloak auth Resource for the SiteUser model in the site. 

In [ ]:
eval $(env SITE_NAMESPACE="$(oc project -q)" ./vars.py)

ACCESS_TOKEN=$(curl -k -s -X POST -d "username=admin" -d "grant_type=password" -d "client_id=admin-cli" \
  -d "password=$(oc -n keycloak get secret/keycloak-initial-admin -o jsonpath={.data.admin-password} | base64 -d)" \
  $AUTH_URL/realms/master/protocol/openid-connect/token | jq -r ".access_token")

curl -k -i -X POST -H "Content-Type: application/json" -H "Authorization: Bearer $ACCESS_TOKEN" \
  "$AUTH_URL/admin/realms/$AUTH_REALM/clients/$AUTH_CLIENT/authz/resource-server/resource" \
  --data '{"name":"SiteUser","displayName":"SiteUser","scopes":["GET","PATCH"]}'

echo DONE

Create a Keycloak auth Policy that represents all users of the AUTH_CLIENT. 

In [ ]:
eval $(env SITE_NAMESPACE="$(oc project -q)" ./vars.py)

ACCESS_TOKEN=$(curl -k -s -X POST -d "username=admin" -d "grant_type=password" -d "client_id=admin-cli" \
  -d "password=$(oc -n keycloak get secret/keycloak-initial-admin -o jsonpath={.data.admin-password} | base64 -d)" \
  $AUTH_URL/realms/master/protocol/openid-connect/token | jq -r ".access_token")

curl -k -i -X POST -H "Content-Type: application/json" -H "Authorization: Bearer $ACCESS_TOKEN" \
  "$AUTH_URL/admin/realms/$AUTH_REALM/clients/$AUTH_CLIENT/authz/resource-server/policy/client" \
  --data '{"id":"'"$AUTH_REALM"'-all-users","name":"'"$AUTH_REALM"'-all-users","description":"all users","clients":["'"$AUTH_CLIENT"'"]}'

echo DONE

Create a Keycloak auth Permission to use the GET and PATCH scope of a SiteUser Resource to all users of the AUTH_CLIENT by default. 

In [ ]:
eval $(env SITE_NAMESPACE="$(oc project -q)" ./vars.py)

ACCESS_TOKEN=$(curl -k -s -X POST -d "username=admin" -d "grant_type=password" -d "client_id=admin-cli" \
  -d "password=$(oc -n keycloak get secret/keycloak-initial-admin -o jsonpath={.data.admin-password} | base64 -d)" \
  $AUTH_URL/realms/master/protocol/openid-connect/token | jq -r ".access_token")

curl -k -i -X POST -H "Content-Type: application/json" -H "Authorization: Bearer $ACCESS_TOKEN" \
  "$AUTH_URL/admin/realms/$AUTH_REALM/clients/$AUTH_CLIENT/authz/resource-server/permission/scope" \
  --data '{"name":"'"$AUTH_REALM"'-SiteUser-all-users-GET-PATCH"
      ,"decisionStrategy":"AFFIRMATIVE"
      ,"description":"SiteUser all client users GET PATCH"
      ,"resources":["SiteUser"]
      ,"policies":["'"$AUTH_REALM"'-all-users"]
      ,"scopes":["'"$AUTH_REALM"'-GET","'"$AUTH_REALM"'-PATCH"]
      }'

echo DONE

## Create SiteAdmin group in Keycloak

Create the SiteAdmin group in your AUTH_REALM. 

In [ ]:
eval $(env SITE_NAMESPACE="$(oc project -q)" ./vars.py)

ACCESS_TOKEN=$(curl -k -s -X POST -d "username=admin" -d "grant_type=password" -d "client_id=admin-cli" \
  -d "password=$(oc -n keycloak get secret/keycloak-initial-admin -o jsonpath={.data.admin-password} | base64 -d)" \
  $AUTH_URL/realms/master/protocol/openid-connect/token | jq -r ".access_token")

curl -k -i -X POST -H "Content-Type: application/json" -H "Authorization: Bearer $ACCESS_TOKEN" \
  "$AUTH_URL/admin/realms/$AUTH_REALM/groups" \
  --data '{"name":"'"$AUTH_SCOPE_ADMIN"'"}'

echo DONE

Create a Keycloak auth Policy that represents the SiteAdmin group. 

In [ ]:
eval $(env SITE_NAMESPACE="$(oc project -q)" ./vars.py)

ACCESS_TOKEN=$(curl -k -s -X POST -d "username=admin" -d "grant_type=password" -d "client_id=admin-cli" \
  -d "password=$(oc -n keycloak get secret/keycloak-initial-admin -o jsonpath={.data.admin-password} | base64 -d)" \
  $AUTH_URL/realms/master/protocol/openid-connect/token | jq -r ".access_token")

AUTH_GROUP_ID=$(curl -k -s -H "Content-Type: application/json" -H "Authorization: Bearer $ACCESS_TOKEN" \
  "$AUTH_URL/admin/realms/$AUTH_REALM/groups?search=$AUTH_SCOPE_ADMIN" | jq -r ".[0].id")

curl -k -i -X POST -H "Content-Type: application/json" -H "Authorization: Bearer $ACCESS_TOKEN" \
  "$AUTH_URL/admin/realms/$AUTH_REALM/clients/$AUTH_CLIENT/authz/resource-server/policy/group" \
  --data '{"id":"'"$AUTH_REALM"'-group-'"$AUTH_SCOPE_ADMIN"'","name":"'"$AUTH_REALM"'-group-'"$AUTH_SCOPE_ADMIN"'","description":"'"$AUTH_SCOPE_ADMIN"' group","groups":["'"$AUTH_GROUP_ID"'"]}'

echo DONE

## Create SuperAdmin group in Keycloak

Create the SuperAdmin group in your AUTH_REALM. 

In [ ]:
eval $(env SITE_NAMESPACE="$(oc project -q)" ./vars.py)

ACCESS_TOKEN=$(curl -k -s -X POST -d "username=admin" -d "grant_type=password" -d "client_id=admin-cli" \
  -d "password=$(oc -n keycloak get secret/keycloak-initial-admin -o jsonpath={.data.admin-password} | base64 -d)" \
  $AUTH_URL/realms/master/protocol/openid-connect/token | jq -r ".access_token")

curl -k -i -X POST -H "Content-Type: application/json" -H "Authorization: Bearer $ACCESS_TOKEN" \
  "$AUTH_URL/admin/realms/$AUTH_REALM/groups" \
  --data '{"name":"'"$AUTH_SCOPE_SUPER_ADMIN"'"}'

echo DONE

Create a Keycloak auth Policy that represents the SiteAdmin group. 

In [ ]:
eval $(env SITE_NAMESPACE="$(oc project -q)" ./vars.py)

ACCESS_TOKEN=$(curl -k -s -X POST -d "username=admin" -d "grant_type=password" -d "client_id=admin-cli" \
  -d "password=$(oc -n keycloak get secret/keycloak-initial-admin -o jsonpath={.data.admin-password} | base64 -d)" \
  $AUTH_URL/realms/master/protocol/openid-connect/token | jq -r ".access_token")

AUTH_GROUP_ID=$(curl -k -s -H "Content-Type: application/json" -H "Authorization: Bearer $ACCESS_TOKEN" \
  "$AUTH_URL/admin/realms/$AUTH_REALM/groups?search=$AUTH_SCOPE_SUPER_ADMIN" | jq -r ".[0].id")

curl -k -i -X POST -H "Content-Type: application/json" -H "Authorization: Bearer $ACCESS_TOKEN" \
  "$AUTH_URL/admin/realms/$AUTH_REALM/clients/$AUTH_CLIENT/authz/resource-server/policy/group" \
  --data '{"id":"'"$AUTH_REALM"'-group-'"$AUTH_SCOPE_SUPER_ADMIN"'","name":"'"$AUTH_REALM"'-group-'"$AUTH_SCOPE_SUPER_ADMIN"'","description":"'"$AUTH_SCOPE_SUPER_ADMIN"' group","groups":["'"$AUTH_GROUP_ID"'"]}'

echo DONE